In [6]:
core_path="C:/Users/n75se/Documents/Myscript/99_work/dummyd_TechCorp/data"
base_path=f'{core_path}/base'
point_path=f'{core_path}/point'
mo_path=f'{core_path}/mobile_order'

# ECサイト事業関連ファイルの読み込み
def add_path(file):
    return f'{point_path}/{file}'

ポイントカード事業に関する分析を行う前に、分析の観点を整理しましょう。提供されたデータには以下の2つのファイルが含まれています：

ポイントカード情報 (point_cards.csv): 顧客が保有するポイントカードに関する情報。
ポイント履歴 (point_history.csv): 顧客によるポイントの獲得と使用の履歴。
これらのデータを基に以下のような分析観点が考えられます：

ポイントの獲得と使用の傾向: 顧客がどのようにポイントを獲得し、使用しているかの傾向を分析します。例えば、ポイントの平均獲得量、平均使用量、獲得から使用までの平均時間など。

ポイントカードの利用状況: ポイントカードを持つ顧客の割合、アクティブなポイントカード保有者の割合、ポイントカードの利用頻度など。

顧客セグメントとポイントの関連: 顧客の属性（年齢、性別、地域など）とポイントの獲得・使用の関連を分析します。

ポイント利用と顧客ロイヤリティ: ポイントを頻繁に使用する顧客とそうでない顧客との間で、購買行動やロイヤリティに違いがあるかを分析します。

これらの分析を通じて、顧客の行動を理解し、ポイントプログラムの効果を評価し、改善策を考えることができます。

まずは、提供されたデータの内容を確認し、上記の観点に基づいて具体的な分析計画を立てます。データの構造を確認してから、分析を進めていきます。

提供されたデータには以下の内容が含まれています：

ポイントカード情報 (point_cards.csv):

card_id: ポイントカードID
user_id: ユーザーID
total_points: 現在のポイント残高
last_updated: 最終更新日
ポイント履歴 (point_history.csv):

history_id: 履歴ID
card_id: ポイントカードID
point_change: ポイント変動量（獲得または使用）
change_date: 変動日
order_id: 関連する注文ID（存在する場合）
order_type: 注文タイプ（例：モバイルオーダー、QRオーダー、ECオーダー）
これらのデータを基に、先ほど挙げた分析観点に沿って分析を進めます。最初の分析として、ポイントの獲得と使用の傾向に関する分析を行い、顧客がどのようにポイントを獲得し、使用しているかの傾向を探ります。

それでは、ポイント履歴データを用いて、ポイントの獲得と使用の傾向に関する分析を開始します。 ​

In [7]:
import pandas as pd

# 各ファイルを読み込む
users_df = pd.read_csv(base_path+"/users.csv")
point_cards_df = pd.read_csv(add_path("point_cards.csv"))
point_history_df = pd.read_csv(add_path("point_history.csv"))
mobile_orders_df = pd.read_csv(mo_path + "/mobile_orders.csv")

# 各データフレームの最初の数行を表示
(point_cards_df.head(), point_history_df.head())


(   card_id  user_id  total_points last_updated
 0        1       20          -796   2023-01-05
 1        2      364           950   2021-03-28
 2        3      123          -327   2022-12-05
 3        4      454          -209   2023-08-22
 4        5       97            99   2023-09-08,
    history_id   card_id   point_change   change_date   order_id     order_type
 0            1        70            -90  2023-11-03         1301  mobile_order
 1            2        24             64  2021-05-19         2048      qr_order
 2            3       267            -13  2022-02-01         1345      qr_order
 3            4       112            166  2023-04-23          477      ec_order
 4            5        27            359  2022-08-10          822      qr_order)

ポイントの獲得と使用の傾向に関する分析結果は以下の通りです：

1. 平均獲得ポイント:
   - 顧客は平均で約499.74ポイントを獲得しています。
2. 平均使用ポイント:
   - 顧客は平均で約497.39ポイントを使用しています。
3. ポイント獲得から使用までの平均時間:
   - 顧客はポイントを獲得してから平均約13.24日で使用しています。

これらの結果から、ポイントカードプログラムは顧客にとって活発に利用されていることがわかります。  
ポイントの獲得量と使用量がほぼ等しく、顧客がポイントを比較的短期間で使用していることが示されています。  
この情報はポイントプログラムの改善やマーケティング戦略の策定に役立つでしょう。

In [10]:
# 列名にスペースが含まれているため、これを修正
point_history_df.columns = point_history_df.columns.str.strip()

# 再度、ポイントの獲得と使用の傾向に関する分析
point_earnings = point_history_df[point_history_df['point_change'] > 0]
point_spendings = point_history_df[point_history_df['point_change'] < 0]

# ポイント獲得と使用の平均値を計算
average_earned_points = point_earnings['point_change'].mean()
average_spent_points = point_spendings['point_change'].mean()

# ポイント獲得から使用までの平均時間を計算
point_history_with_order = point_history_df.merge(mobile_orders_df[['order_id', 'order_date']], on='order_id', how='left')
point_history_with_order['change_date'] = pd.to_datetime(point_history_with_order['change_date'])
point_history_with_order['order_date'] = pd.to_datetime(point_history_with_order['order_date'])
point_history_with_order['time_to_use'] = (point_history_with_order['change_date'] - point_history_with_order['order_date']).dt.days

average_time_to_use = point_history_with_order[point_history_with_order['point_change'] < 0]['time_to_use'].mean()

(average_earned_points, average_spent_points, average_time_to_use)


(499.7424639580603, -497.3892663043478, 13.239809782608695)

次は、「ポイントカードの利用状況」に関する分析を行います。この分析では、以下の点に着目します：  

1. ポイントカードを持つ顧客の割合。
2. アクティブなポイントカード保有者の割合（最近一定期間内にポイントの変動があったカードの割合）。
3. ポイントカードの利用頻度（顧客がポイントを獲得または使用する頻度）。

これらの分析を通じて、ポイントカードプログラムの普及度と効果を評価し、改善のための洞察を得ることができます。  

  

「ポイントカードの利用状況」に関する分析結果は以下の通りです：  
1. ポイントカードを持つ顧客の割合:
   - 全顧客の約45.4%がポイントカードを保有しています。
2. アクティブなポイントカード保有者の割合:
   - 全ポイントカード保有者の約35.33%が過去1年以内にアクティブ（ポイント変動があった）でした。
3. ポイントカードの利用頻度:
   - ポイントカードを持つ顧客は平均して10回ポイントを獲得または使用しています。

これらの結果から、ポイントカードプログラムは一定の普及を見せていますが、全顧客の中での保有率やアクティブ率は改善の余地があります。  
また、平均利用頻度は比較的高く、アクティブなユーザーは積極的にポイントを利用していることが分かります。  

In [11]:
from datetime import datetime

# 現在の日付を取得（分析時点）
current_date = datetime.now()

# ポイントカードの総数とアクティブなカードの数を計算
total_cards = len(point_cards_df)

# ポイントカードを持つ顧客の割合
total_users = len(users_df)
users_with_cards = point_cards_df['user_id'].nunique()
percentage_users_with_cards = (users_with_cards / total_users) * 100

# 'last_updated' 列を日付型に変換
point_cards_df['last_updated'] = pd.to_datetime(point_cards_df['last_updated'])

# 再計算
active_cards = point_cards_df[point_cards_df['last_updated'] >= (current_date - pd.DateOffset(years=1))].shape[0]

# ポイントカードを持つ顧客の割合、アクティブなポイントカード保有者の割合、ポイントカードの利用頻度の再計算
percentage_users_with_cards = (users_with_cards / total_users) * 100
percentage_active_cards = (active_cards / total_cards) * 100
# ポイントカードの利用頻度（ポイント変動の平均回数）
average_usage_frequency = point_history_df['card_id'].value_counts().mean()

(percentage_users_with_cards, percentage_active_cards, average_usage_frequency)


(45.4, 35.333333333333336, 10.0)

次は、「顧客セグメントとポイントの関連」に関する分析を行います。  
この分析では、顧客の属性（年齢、性別など）とポイントの獲得・使用の関連を調査します。具体的には、以下の点に焦点を当てます：  
1. 顧客の年齢層別のポイント獲得・使用の傾向。
2. 顧客の性別によるポイント獲得・使用の違い。
3. 地理的な場所や他のデモグラフィック要素に基づくポイントの利用傾向。
   
これらの分析を通じて、特定の顧客セグメントがポイントプログラムをどのように利用しているかを理解し、それに応じたマーケティング戦略やプログラム改善の方向性を見出します。  

それでは、「顧客セグメントとポイントの関連」に関する分析を開始します。まずは、顧客の年齢層別と性別に焦点を当てて分析を進めます。  


「顧客セグメントとポイントの関連」に関する分析結果は以下の通りです：  
1. 年齢層別のポイント獲得・使用の傾向:
   - 例えば、18歳の顧客は平均で約527.50ポイントを獲得し、約-552.33ポイントを使用しています。
   - 19歳の顧客は平均で約577.00ポイントを獲得し、約-101.50ポイントを使用しています。
   - 年齢が上がるにつれて、ポイントの獲得・使用量にバラツキが見られます。
2. 性別別のポイント獲得・使用の傾向:
   - 女性顧客は平均で約538.47ポイントを獲得し、約-535.77ポイントを使用しています。
   - 男性顧客は平均で約485.41ポイントを獲得し、約-500.52ポイントを使用しています。

これらの結果から、年齢層や性別によってポイントの獲得・使用の傾向に違いがあることがわかります。  
特に若年層ではポイントの使用が活発であることが示されています。  
また、性別による大きな差は見られませんが、女性顧客の方が若干ポイントを多く獲得・使用している傾向があります。  

これらの洞察は、ターゲットとなる顧客セグメントに合わせたポイントプログラムの改善やマーケティング戦略の策定に役立ちます。  

In [ ]:
# 顧客の年齢層別と性別に基づくポイントの獲得・使用の分析
# ポイントカードデータとユーザーデータを結合
merged_points_users_df = pd.merge(point_cards_df, users_df, on='user_id')

# 年齢層別のポイント獲得・使用の平均値を計算
age_group_points = merged_points_users_df.groupby('age').agg(
    average_earned_points=pd.NamedAgg(column='total_points', aggfunc=lambda x: x[x > 0].mean()),
    average_spent_points=pd.NamedAgg(column='total_points', aggfunc=lambda x: x[x < 0].mean())
).reset_index()

# 性別別のポイント獲得・使用の平均値を計算
gender_points = merged_points_users_df.groupby('gender').agg(
    average_earned_points=pd.NamedAgg(column='total_points', aggfunc=lambda x: x[x > 0].mean()),
    average_spent_points=pd.NamedAgg(column='total_points', aggfunc=lambda x: x[x < 0].mean())
).reset_index()

(age_group_points.head(), gender_points.head())


次に、「ポイント利用と顧客ロイヤリティ」に関する分析を行います。  
この分析では、ポイントを頻繁に使用する顧客とそうでない顧客の間で、購買行動やロイヤリティに違いがあるかを調査します。  
具体的には以下の点に焦点を当てます：  

1. ポイントを頻繁に使用する顧客（例えば、上位25%）とそうでない顧客（例えば、下位25%）の購買頻度や平均購入金額の比較。
2. ポイントの使用頻度に基づく顧客のセグメンテーションと、それぞれのセグメントにおける購買行動の特徴。

これらの分析を通じて、ポイントプログラムが顧客のロイヤリティや購買行動にどのように影響しているかを理解し、より効果的な顧客エンゲージメント戦略を策定することができます。  


「ポイント利用と顧客ロイヤリティ」に関する分析結果は以下の通りです：  
1. ポイントを頻繁に使用する顧客（上位25%）:
   - 平均購買頻度: 約6.21回
   - 平均購入金額: 約5024.84円
2. ポイントをあまり使用しない顧客（下位25%）:
   - 平均購買頻度: 約6.17回
   - 平均購入金額: 約5033.62円

これらの結果から、ポイントを頻繁に使用する顧客とそうでない顧客間での購買行動に大きな違いは見られません。  
両グループの平均購買頻度と平均購入金額はほぼ同等です。  

この分析から、ポイントプログラムの使用頻度が顧客の購買行動に直接的な大きな影響を与えているわけではないことが示唆されます。  
ただし、ポイントプログラムが顧客のロイヤリティや他の非金銭的な価値にどのように影響しているかについては、さらなる分析が必要かもしれません。  